# LLMOps for Production RAG

Welcome to the LLMOps for Production RAG workshop! In this workshop, we will cover:

1. Creating a baseline RAG pipeline
2. Bootstrapping an evaluation dataset
3. RAG Hyperparameter Optimization


In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/unionai-oss/llmops-production-rag.git
    %cd llmops-production-rag
    %pip install -r requirements.lock.txt

In [ ]:
%cd /content/llmops-production-rag
!union create login --auth device-flow --serverless

## Creating a Baseline RAG Pipeline

Create the vector store:

```bash
union run --remote llmops_rag/vector_store.py create --include_union --exclude_patterns '["/api/", "/_tags/"]'
```

Then run the simple rag pipeline

```bash
unoin run --remote llmops_rag/simple_rag.py run --question "Write a simple hello world workflow"
```

## Bootstrapping an Evaluation Dataset

First create a raw knowledge base:

```bash
union run --remote llmops_rag/vector_store.py create_knowledge_base --include_union --exclude_patterns '["/api/", "/_tags/"]'
```

Then synthesize a question and answer dataset:

```bash
union run --remote llmops_rag/synthesize_data.py run \
   --n_questions 1 \
   --n_answers 5
```

Register the data annotation workflow:

```bash
union register llmops_rag/annotate_data.py
```

Run a single annotation session to test it out:

```bash
union run --remote llmops_rag/annotate_data.py create_annotation_set --random_seed 42 --n_samples 10
```

### Annotator App

Create a `secrets.txt` file to store these credentials. This file is ignored by
git and should look something like this:

```
UNIONAI_SERVERLESS_API_KEY=<UNIONAI_SERVERLESS_API_KEY>
```

Export the secrets to your environment:

```bash
export $(cat secrets.txt | xargs)
```

Run the app

```bash
streamlit run streamlit/app.py
```

## RAG Hyperparameter Optimization

Create the eval dataset:

```bash
union run --remote llmops_rag/eval_dataset.py create --min_annotations_per_question 1
```

Evaluate a RAG experiment:

```bash
union run --remote llmops_rag/hpo.py run --eval_configs config/eval_inputs_prompt.yaml
```

Experiment with different chunksizes:

```bash
union run --remote llmops_rag/hpo.py run --eval_configs config/eval_inputs_chunksize.yaml
```

Experiment with different splitters:

```bash
union run --remote llmops_rag/hpo.py run --eval_configs config/eval_inputs_splitter.yaml
```